In [4]:
from pydriller import Repository
from pprint import pprint

In [10]:
with open('commits.txt', 'w') as file:
  for commit in Repository('https://github.com/apache/seatunnel.git').traverse_commits():
    for f in commit.modified_files:
      file.write(f.filename + ' ')
    file.write('\n')

In [11]:
def is_test_file(filename):
    return filename.endswith("Test.java")

def is_class_file(filename):
    return not filename.endswith("Test.java") and filename.endswith(".java")

def get_class_file_name(test_filename):
    return test_filename.replace("Test", "")

def get_test_file_name(class_filename):
    return class_filename.replace(".java", "Test.java")

In [12]:
def tdd_check(commits):
    visited_test_files = []
    visited_class_files = []
    tdd_set = []
    tdd = 0
    non_tdd = 0
    same = 0

    with open(commits, "r") as files:
        for line in files:
            changed_files = line.split()
            for file in changed_files:
                if is_test_file(file) and file not in visited_test_files:
                    if get_class_file_name(file) in visited_class_files:
                        non_tdd += 1
                    elif get_class_file_name(file) in changed_files:
                        same += 1
                    visited_test_files.append(file)
                if is_class_file(file) and file not in visited_class_files:
                    if get_test_file_name(file) in visited_test_files:
                        tdd += 1
                        tdd_set.append(file)
            # increase visited class files after a line is read
            for file in changed_files:
                if is_class_file(file):
                    visited_class_files.append(file)

    print("TDD: ", tdd)
    print("Non_TDD: ", non_tdd)
    print("Same: ", same)
    print(tdd_set)


In [13]:
tdd_check("commits.txt")

TDD:  3
Non_TDD:  70
Same:  67
['FlinkEnvParameterParser.java', 'LogicalDagGenerator.java', 'HadoopLoginFactory.java']
